In [4]:
import os
import pathlib
import pprint
import json
import itertools
import shutil

In [5]:
def save_json_dict(
        json_filename,
        json_dict
        ):

    with open(json_filename, 'w') as json_file:
        json.dump(json_dict, json_file, indent=4)


def load_json_dict(
        json_filename
        ):

    with open(json_filename, 'r') as json_file:
        json_dict = json.load(json_file)

    return json_dict

---

In [6]:
cutdown_ratio = 0.020

#

directory_list = [
    "efficientnet_v2_s",
    "swint_v2_t",
    "cvt_21-384-22k"
]

backbone_list = [
    "EfficientNetV2SmallBackbone",
    "SwinTransformerV2TinyBackbone",
    "CvTransformerB21I384D22kBackbone"
]

gpu_idxs_dict = {
    "efficientnet_v2_s": {
        1: [0],
        2: [1, 2],
        3: [0, 1, 2],
        4: [0, 1, 2, 3]
    },
    "swint_v2_t": {
        1: [0],
        2: [1, 2],
        3: [0, 1, 2],
        4: [0, 1, 2, 3]
    },
    "cvt_21-384-22k": {
        1: [0],
        2: [1, 2],
        3: [0, 1, 2],
        4: [0, 1, 2, 3]
    }
}

img_size_list = [256, 384]

batch_size_list = [4, 8, 16, 32, 64]

num_gpus_list = [1, 2, 3, 4]

autocast_list = [True, False]

num_workers_list = [2, 4, 8]

#

num_changes = 0

params_dir = os.path.join(pathlib.Path.home(), "fashion_retrieval", "params", "ret_test_DDP_2")
#if os.path.exists(params_dir): shutil.rmtree(params_dir)
#os.mkdir(params_dir)

for directory, backbone in \
    zip(directory_list, backbone_list):

    params_subdir = os.path.join(pathlib.Path.home(), "fashion_retrieval", "params", "ret_test_DDP_2", directory)
    #if os.path.exists(params_subdir): shutil.rmtree(params_subdir)
    #os.mkdir(params_subdir)

    for exp_idx, (img_size, batch_size, num_gpus, autocast, num_workers) in\
        enumerate(itertools.product(img_size_list, batch_size_list, num_gpus_list, autocast_list, num_workers_list)):

        num_changes += 1

        template_params_filename = os.path.join(pathlib.Path.home(), "fashion_retrieval", "params", "templates", "ret_train_DDP_new.json")
        params = load_json_dict(template_params_filename)

        #

        params["experiment_name"] = os.path.join("ret_test_DDP_2", directory, "test_{:03d}".format(exp_idx))
        params["settings"]["device_idxs"] = gpu_idxs_dict[directory][num_gpus]
        params["settings"]["backbone"]["class"] = backbone
        params["settings"]["backbone"]["img_size"] = img_size
        params["settings"]["data_loading"]["cutdown_ratio"] = cutdown_ratio
        params["settings"]["stage_1"]["data_loading"]["batch_size"] = batch_size
        params["settings"]["stage_1"]["data_loading"]["num_workers"] = num_workers
        params["settings"]["stage_1"]["autocast"]["enabled"] = autocast
        params["settings"]["stage_2"]["data_loading"]["batch_size"] = batch_size
        params["settings"]["stage_2"]["data_loading"]["num_workers"] = num_workers
        params["settings"]["stage_2"]["autocast"]["enabled"] = autocast
        params["settings"]["test"]["data_loading"]["batch_size"] = batch_size
        params["settings"]["test"]["data_loading"]["num_workers"] = num_workers

        #

        dest_params_filename = os.path.join(pathlib.Path.home(), "fashion_retrieval", "params", "ret_test_DDP_2", directory, "test_{:03d}__train_ret_DDP_preview.json".format(exp_idx))
        save_json_dict(dest_params_filename, params)

print(num_changes)


720
